In [1]:
import pandas as pd
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import mlflow

from prefect import flow, task

@task
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

@task
def add_features(df_train, df_val):
    # df_train = read_dataframe(train_path)
    # df_val = read_dataframe(val_path)

    print(len(df_train))
    print(len(df_val))

    df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
    df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    target = 'duration'
    y_train = df_train[target].values
    y_val = df_val[target].values

    return X_train, X_val, y_train, y_val, dv

@task
def train_model_search(train, valid, y_val):
    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag("model", "xgboost")
            mlflow.log_params(params)
            booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=100,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
            y_pred = booster.predict(valid)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:linear',
        'seed': 42
    }

    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=1,
        trials=Trials()
    )
    return

@task
def train_best_model(train, valid, y_val, dv):
    with mlflow.start_run():

        best_params = {
            'learning_rate': 0.09585355369315604,
            'max_depth': 30,
            'min_child_weight': 1.060597050922164,
            'objective': 'reg:linear',
            'reg_alpha': 0.018060244040060163,
            'reg_lambda': 0.011658731377413597,
            'seed': 42
        }

        mlflow.log_params(best_params)

        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        with open("model/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("model/preprocessor.b", artifact_path="preprocessor")

        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")



In [7]:
@flow
def main(train_path: str="./data/green_tripdata_2022-02.parquet",
        val_path: str="./data/green_tripdata_2022-01.parquet"):
    MLFLOW_TRACKING_URI = "http://localhost:5000"
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment("nyc-taxi-experiment")
    X_train = read_dataframe(train_path)
    X_val = read_dataframe(val_path)
    X_train, X_val, y_train, y_val, dv = add_features(X_train, X_val)
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    train_model_search(train, valid, y_val)
    train_best_model(train, valid, y_val, dv)

/home/forkbo/p3_envs/env1/lib/python3.8/site-packages/prefect/flows.py:206: UserWarning: A flow named 'main' and defined at '/tmp/ipykernel_7001/3414888103.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [8]:
main()

16:42:30.764 | INFO    | prefect.engine - Created flow run 'garnet-woodlouse' for flow 'main'

16:42:31.112 | INFO    | Flow run 'garnet-woodlouse' - Created task run 'read_dataframe-0' for task 'read_dataframe'

16:42:31.114 | INFO    | Flow run 'garnet-woodlouse' - Executing 'read_dataframe-0' immediately...

16:42:31.500 | INFO    | Task run 'read_dataframe-0' - Finished in state Completed()

16:42:31.563 | INFO    | Flow run 'garnet-woodlouse' - Created task run 'read_dataframe-1' for task 'read_dataframe'

...

[16:42:48] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


[0]	validation-rmse:15.07692
[1]	validation-rmse:13.76811
[2]	validation-rmse:12.60608
[3]	validation-rmse:11.57898
[4]	validation-rmse:10.67389
[5]	validation-rmse:9.88104
[6]	validation-rmse:9.18852
[7]	validation-rmse:8.58618
[8]	validation-rmse:8.07301
[9]	validation-rmse:7.63151
[10]	validation-rmse:7.25677
[11]	validation-rmse:6.93943
[12]	validation-rmse:6.68086
[13]	validation-rmse:6.45915
[14]	validation-rmse:6.27859
[15]	validation-rmse:6.13557
[16]	validation-rmse:6.01482
[17]	validation-rmse:5.91964
[18]	validation-rmse:5.84447
[19]	validation-rmse:5.78405
[20]	validation-rmse:5.74105
[21]	validation-rmse:5.70337
[22]	validation-rmse:5.67429
[23]	validation-rmse:5.65410
[24]	validation-rmse:5.63817
[25]	validation-rmse:5.62572
[26]	validation-rmse:5.61880
[27]	validation-rmse:5.61323
[28]	validation-rmse:5.60971
[29]	validation-rmse:5.60822
[30]	validation-rmse:5.60697
[31]	validation-rmse:5.60723
[32]	validation-rmse:5.60755
[33]	validation-rmse:5.60853
[34]	validation-rms

16:43:01.332 | INFO    | Task run 'train_best_model-0' - Finished in state Completed()

16:43:09.942 | INFO    | Flow run 'garnet-woodlouse' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=       VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
 0             2  2022-02-01 00:20:21   2022-02-01 00:24:30                  N   
 1             2  2022-02-01 00:32:26   2022-02-01 00:35:31                  N   
 2             1  2022-02-01 00:17:27   2022-02-01 00:44:44                  N   
 3             2  2022-02-01 00:45:37   2022-02-01 01:27:16                  N   
 4             2  2022-02-01 00:06:46   2022-02-01 00:30:06                  N   
 ...         ...                  ...                   ...                ...   
 69394         2  2022-02-28 23:27:00   2022-02-28 23:38:00               None   
 69395         2  2022-02-28 23:59:00   2022-03-01 00:10:00               None   
 69396         2  2022-02-28 23:18:00   2022-02-28 23:27:00               None   
 69397         2  2022-02-28 23:31:00   2022-02-28 23:39:00               None   
 69398         2  2022-02-28 23:14:00   2022-02-28 